In [1]:
#Load libraries
import pandas as pd
from configobj import ConfigObj
import os
import sys
import math
import sqlalchemy

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from functions.transport_by_tower_owner import transport_by_tower_owner
from functions.utils import print_full, elevation_google, database

In [3]:
## define paths and variables

config_path = "/home/jovyan/shared/rural_planner_refactoring/config_files/config_br"

parser = ConfigObj(config_path)

sql_path =  parser['sql_path']
country_folder = parser['country_folder']
schema = parser['transport_multihop_params']['schema']


output_table = parser['transport_multihop_params']['output_table']
temporary_table = parser['transport_multihop_params']['temporary_table']
table_towers = parser['transport_by_tower_params']['table_infrastructure']
table_clusters = parser['clustering_params']['output_table']
table_clusters_3g = parser['clustering_3g_params']['output_table']
table_settlements = parser['clustering_params']['table_settlements']
table_transport = parser['transport_by_tower_params']['output_table']
table_transport_clusters = parser['transport_gf_cl_params']['output_table']

sources_omit = parser['transport_by_tower_params']['sources_omit']
owners_tx = dict(parser['transport_by_tower_params']['owners_tx'])

third_party_owners = parser['transport_by_tower_params']['third_party_owners']
regional_owners = parser['transport_by_tower_params']['regional_owners']
tef_alias = (owners_tx.keys() - (regional_owners + third_party_owners)).pop()

fiber_radius = int(parser['transport_by_tower_params']['fiber_radius']) 

In [10]:
## Get viable intermediate hops for clusters with no tx (not based on owner, just one set of columns)

owners = ''
owner_name = 'intermediate_hops'
query_path = sql_path + '/' + country_folder + '/' + 'transport_multihop_load_inputs.sql'
query_path_all = sql_path + '/' + country_folder + '/' + 'transport_multihop_load_all_inputs.sql'

df_output = transport_by_tower_owner(parser, owners, owner_name, query_path, query_path_all, column_name = 'centroid')

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
 [SQL: "SELECT * FROM (\n    SELECT S1.centroid,\n    S1.latitude AS latitude_1,\n    S1.longitude AS longitude_1,\n    S1.tower_height as tower_height_1,\n    S2.tower_id AS tower_id_2,\n    S2.latitude AS latitude_2,\n    S2.longitude AS longitude_2,\n    S2.tower_height AS tower_height_2,\n    S2.source AS owner_2,\n    ST_Distance(S1.geom, ST_Transform(S2.geom,3857)) AS distance\n    FROM (SELECT A.centroid,\n    COALESCE(I.latitude, S.latitude) AS latitude,\n    COALESCE(I.longitude, S.longitude) AS longitude,\n    COALESCE(I.tower_height, 50) AS tower_height,\n    ST_Transform(COALESCE(I.geom, S.geom),3857) AS geom\n    FROM (SELECT * FROM rural_planner_dev.clusters) A\n    LEFT JOIN rural_planner_dev.infrastructure_global i\n    ON A.centroid=i.tower_id::text\n    LEFT JOIN rural_planner_dev.settlem

In [84]:
#Re-shape data frames
#Round doubles
df_output.additional_height_tower_1_intermediate_hop = df_output.additional_height_tower_1_intermediate_hop.round()
df_output.additional_height_tower_2_intermediate_hop = df_output.additional_height_tower_2_intermediate_hop.round()

In [85]:
#Export output

query_path = sql_path + '/' + country_folder + '/' + 'transport_multihop_add_tx.sql'

with open(query_path) as file, database(config_path) as db:    
    df_output.to_sql(temporary_table, con=db, if_exists = 'replace', schema = schema, index = False, 
                dtype = {'centroid': sqlalchemy.types.String(),
                         'intermediate_hop_id': sqlalchemy.types.Integer(),
                         'distance_intermediate_hop_m': sqlalchemy.types.Integer(),
                         'line_of_sight_intermediate_hop': sqlalchemy.types.Boolean(),
                         'additional_height_tower_1_intermediate_hop_m': sqlalchemy.types.Integer(),
                         'additional_height_tower_2_intermediate_hop_m': sqlalchemy.types.Integer()
                        })
    query = file.read()
    query_formatted = query.format(schema = schema,
                                   output_table = output_table,
                                   temporary_table = temporary_table,
                                   transport_table = transport_table,
                                   tef_alias = tef_alias)
    db.execute(query_formatted)